In [1]:
from agential.cog.agent.critic import CriticAgent
from langchain_community.tools.google_serper.tool import GoogleSerperRun
from langchain_community.utilities.google_serper import GoogleSerperAPIWrapper
from langchain_community.utilities.google_search import GoogleSearchAPIWrapper
from langchain_community.chat_models.openai import ChatOpenAI
from agential.cog.prompts.critic import (
    # Prompts.
    CRITIC_POT_INSTRUCTION_GSM8K,
    CRITIC_CRITIQUE_INSTRUCTION_GSM8K, 
    CRITIC_CRITIQUE_NO_TOOL_INSTRUCTION_GSM8K,
    # Examples.
    GSM8K_FEWSHOT_EXAMPLES_POT,
    GSM8K_FEWSHOT_EXAMPLES_CRITIC,
    GSM8K_FEWSHOT_EXAMPLES_CRITIC_NO_TOOL,
)


import warnings
warnings.filterwarnings("ignore")

import dotenv

dotenv.load_dotenv()

llm = ChatOpenAI()
search = GoogleSearchAPIWrapper()

In [ ]:
from agential.cog.functional.critic import _prompt_agent

question = "Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?"

code = _prompt_agent(
    llm=llm, question=question, examples=GSM8K_FEWSHOT_EXAMPLES_POT, prompt=CRITIC_POT_INSTRUCTION_GSM8K
)

In [ ]:
from agential.cog.functional.critic import _prompt_critique
question = "Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?"

code = """clips_sold_in_april = 48
clips_sold_in_may = clips_sold_in_april / 30
answer = clips_sold_in_april + clips_sold_in_may"""

critique = _prompt_critique(
    llm=llm,
    question=question,
    examples=GSM8K_FEWSHOT_EXAMPLES_CRITIC,
    answer=code,
    critique="",
    additional_keys={
        "execution_status": "Done",
        "code_answer": "64"
    },
    prompt=CRITIC_CRITIQUE_INSTRUCTION_GSM8K,
).split("Here's")[0]

In [ ]:
responses = [
    code,
    "The code provided is incorrect.\n\nThe issue lies in the calculation of the number of clips sold in May. It is stated that Natalia sold half as many clips in May compared to April, which means she sold 48/2 = 24 clips in May. The code incorrectly calculates the number of clips sold in May by dividing the number of clips sold in April by 30.\n\nHere's the corrected version of the code:\n\n```python\nclips_sold_in_april = 48\nclips_sold_in_may = clips_sold_in_april / 2  # Natalia sold half as many clips in May\nanswer = clips_sold_in_april + clips_sold_in_may\n```"
]

In [ ]:
better = _prompt_critique(
    llm=llm,
    question=question,
    examples=GSM8K_FEWSHOT_EXAMPLES_CRITIC,
    answer=code,
    critique=critique + "\n\n" + "Here's a better solution:\n```python\n",
    additional_keys={
        "execution_status": "Done",
        "code_answer": "64"
    },
    prompt=CRITIC_CRITIQUE_INSTRUCTION_GSM8K,
).split("```")[0]

In [ ]:
print(better)

In [ ]:
"""
         | with tool | without tool |
PoT      |    o      |      o       |
"""

In [ ]:
agent = CriticAgent(llm=llm, mode="search", search=search)

In [7]:
question = "Janet's ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with 4933828. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?"
# question = 'Who was once considered the best kick boxer in the world, however he has been involved in a number of controversies relating to his "unsportsmanlike conducts" in the sport and crimes of violence outside of the ring'

import warnings
warnings.filterwarnings("ignore")

agent = CriticAgent(llm=llm, mode="code_interpreter")
out = agent.generate(
    question=question,
    examples=GSM8K_FEWSHOT_EXAMPLES_POT,
    prompt=CRITIC_POT_INSTRUCTION_GSM8K,
    critique_examples=GSM8K_FEWSHOT_EXAMPLES_CRITIC,
    critique_prompt=CRITIC_CRITIQUE_INSTRUCTION_GSM8K,
    max_interactions=2
)

'total_eggs = 16\neaten_eggs = 3\nbaked_eggs = 4933828\nsold_eggs = total_eggs - eaten_eggs - baked_eggs\ndollars_per_egg = 2\nanswer = sold_eggs * dollars_per_egg'
"The problem with the above code is that the calculation for the number of eggs sold is incorrect. \n\nLet's correct the code:\n\n```python\ntotal_eggs = 16\neaten_eggs = 3\nbaked_eggs = 4933828\nsold_eggs = total_eggs - eaten_eggs - baked_eggs\ndollars_per_egg = 2\n\n# Make sure the number of sold eggs is non-negative\nif sold_eggs < 0:\n    sold_eggs = 0\n\nanswer = sold_eggs * dollars_per_egg\n```"
'total_eggs = 16\neaten_eggs = 3\nbaked_eggs = 4933828\nsold_eggs = total_eggs - eaten_eggs - baked_eggs\n\n# Check if the number of sold eggs is negative and correct it\nif sold_eggs < 0:\n    sold_eggs = 0\n\ndollars_per_egg = 2\ntotal_money_made = sold_eggs * dollars_per_egg\n\nanswer = total_money_made'
'It is correct.'


2